There is a voluminous literature about how to measure causal impact using the differences-in-differences technique. It includes multiple controversies over how the technique can be applied to panel data with the problems of time-varying heterogeneity. These are fraught and varied issues, and we'll elide them for now. 

Instead we'll look at one of the earliest analyses in the literature, popularised by Card and Krueger in 1994. 


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
from stargazer.stargazer import Stargazer

## The Data

The data from the Card and Krueger study reports measures of employement by location in bordering states of New Jersey and Pennsylvania afer April 1st, 1992 when New Jersey’s minimum wage rose from $4.25 to $5.05 per hour. The interest is in the effect of the different state policies relating to minimum wage. There is juncture in time after which New Jersey enacts a minimum wage policy which is anticipated to have a meaningful negative impact on employment figures. 


In [ ]:
# Load the Data from the minumum wage study
df_ck = pd.read_csv('CK1994.txt', sep='\t')

## Calculate the price of an average meal.
df_ck['price'] = df_ck[['pricesoda', 'pricefry', 'priceentree']].sum(axis=1)

# Count of employees
df_ck['employees'] = df_ck.apply(lambda x: x['empft'] + x['nmgrs'] + 0.5*x['emppt'], axis=1)

# Interaction of state and time for use in OLS
df_ck['treatment'] = df_ck['state']*df_ck['time']

# Group by Region
df_ck['group'] = np.where(df_ck['southj'], 'NJ south',
                    np.where(df_ck['centralj'], 'NJ central', 
                      np.where(df_ck['northj'], 'NJ North', 
                        np.where(df_ck['pa1'], 'PA 1', 
                          np.where(df_ck['pa2'], 'PA 2', np.nan)))))


df_ck.head()

The crucial results reported in the paper show a surprising subversion of expecation. The idea is that the neigbouring states should have comparable working conditions and incentives to employment up until the initiative of the policy change. Hence, it is argued that the difference between the states before and after the change can be a gauge of the causal impact of that policy. The data they looked at surveyed 410 fast-food restaurants in New Jersey and eastern Pennsylvania before and after the rise in minimum wage for New Jersey.


In [ ]:
pivot = (df_ck[['state', 'time', 'employees']].dropna()
  .groupby(['state', 'time'])[['employees']].mean()
  .reset_index()
  .pivot(index='time', columns='state', values='employees')
  )
pivot = pivot.append(pivot.iloc[1] - pivot.iloc[0], ignore_index=True)
pivot.columns = ['PA', 'NJ']
pivot['Diff'] = pivot['NJ'] - pivot['PA']
pivot.index = ['Before', 'After', 'Diff']
pivot

The result here is, by traditional economic logic, surprising in that they "find no indication that the rise in the minimum wage reduced employment." The above table reports the raw differences in average employment per restaurant. That's it. That's the quasi-experiemntal design that launched a thousand imitations. 


In [ ]:
fig, ax = plt.subplots()
ax.plot(['Before', 'After'], [pivot.iloc[0]['PA'],  pivot.iloc[1]['PA']], label='Pennsylvania')
ax.plot(['Before', 'After'], [pivot.iloc[0]['NJ'],  pivot.iloc[1]['NJ']],label='New Jersey')
ax.plot(['Before', 'After'], [pivot.iloc[0]['NJ'],  pivot.iloc[2]['PA'] +pivot.iloc[0]['NJ']], '--', color='darkorange', label='New Jersey Counterfactual')
ax.legend()
ax.set_title("Visualising the Counterfactual");

## A Persuasive Design

It's not just a simple table. The paper was compelling precisely because the design was persuasive, and the Pennsylvania's future is a plausible representation of New Jersey's counterfactual future. 


In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 7))
axs = axs.flatten()
before = df_ck[df_ck['time'] == 0]
after = df_ck[df_ck['time'] == 1]
axs[0].hist(before[before['state'] == 0]['wage_st'], alpha=0.4, bins=20, density=True, ec='black', label='Pennsylvania Before')
axs[0].hist(before[before['state'] == 1]['wage_st'], alpha=0.4, bins=20, density=True, ec='black', label='New Jersey Before')
axs[0].set_xlabel("Wage per Hour in $")
axs[0].legend()
axs[1].hist(after[after['state'] == 0]['wage_st'], alpha=0.4, bins=15, density=True, ec='black', label='Pennsylvania After')
axs[1].hist(after[after['state'] == 1]['wage_st'], alpha=0.4, bins=15, density=True, ec='black', label='New Jersey After')
axs[1].set_xlabel("Wage per Hour in $")
axs[1].legend()
axs[1].set_title("Wage Distribution After")
axs[0].set_title("Wage Distribution Before");

And the corressponding view for the employment figures shows that the difference between before and after periods, for both states are centered around zero. 


In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(12, 5))
axs = axs.flatten()
before = df_ck[df_ck['time'] == 0]
after = df_ck[df_ck['time'] == 1]
axs[0].hist(before[before['state'] == 0]['employees'], alpha=0.4, bins=20, density=True, ec='black', label='Pennsylvania Before')
axs[0].hist(before[before['state'] == 1]['employees'], alpha=0.4, bins=20, density=True, ec='black', label='New Jersey Before')
axs[0].set_xlabel("Employees")
axs[0].legend()
axs[2].hist(after[after['state'] == 0]['employees'], alpha=0.4, bins=15, density=True, ec='black', label='Pennsylvania After')

diff_p = after[after['state'] == 0][['store', 'employees']].merge(before[before['state'] == 0][['store', 'employees']], left_on='store', right_on='store')

diff_nj = after[after['state'] == 1][['store', 'employees']].merge(before[before['state'] == 1][['store', 'employees']], left_on='store', right_on='store')

axs[1].hist(diff_p['employees_x'] - diff_p['employees_y'], alpha=0.4, bins=15, density=True, ec='black', label='Pennsylvania Diff')
axs[1].hist(diff_nj['employees_x'] - diff_nj['employees_y'], alpha=0.4, bins=15, density=True, ec='black', label='NJ Diff')
axs[1].set_xlabel("Before - After")
axs[1].legend()

axs[2].hist(after[after['state'] == 1]['employees'], alpha=0.4, bins=15, density=True, ec='black', label='New Jersey After')
axs[2].set_xlabel("Employees")
axs[2].legend()
axs[2].set_title("Employed Distribution After")
axs[1].set_title("Difference of Employed Distribution")
axs[0].set_title("Employed Distribution Before");


## Robustness to Controls

The robustness of the effect might, in principle, be moderated or refined by other factors. So it's worth exploring the parameter fits for a variety of models. First we recover the simple differences-in-differences control using, and then for other subsequent moodels we add controls for the the location, food chain and whether the restaurant is co-owned. All models recover effectively the same estimate for the interaction term of state over time, which is our differences-in-differences estimate. 


In [ ]:
temp = df_ck[['employees', 'northj', 'centralj', 'pa1', 'pa2', 'time', 'treatment', 'chain', 'state', 'co_owned']].dropna()
temp[['chain_1', 'chain_2', 'chain_3', 'chain_4']] = pd.get_dummies(temp['chain'])
model_0 = smf.ols(formula='employees ~ state + time + treatment', data=temp).fit()
model_1 = smf.ols(formula='employees ~ state + time + chain_1 + chain_2 + chain_3 + treatment', data=temp).fit()
model_2 = smf.ols(formula='employees ~ centralj + pa1 + pa2 + time + treatment', data=temp).fit()

model_3 = smf.ols(formula='employees ~ centralj + pa1 + pa2 + time + chain_1 + chain_2 + chain_3 + treatment', data=temp).fit()

model_4 = smf.ols(formula='employees ~ centralj + pa1 + pa2 + time + chain_1 + chain_2 + chain_3 + co_owned + treatment', data=temp).fit()

stargazer = Stargazer([model_0, model_1, model_2, model_3])
stargazer.render_html()

The effect is consisteny across the model specification. 